In [2]:
import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns
# Render the figure in a notebook:
%matplotlib inline  

from scipy import stats

from mlxtend.evaluate import BootstrapOutOfBag

# You are not loading the entire sklearn package, but only the classes required (there is a lot here!).

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree
from xgboost import XGBRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import learning_curve
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import learning_curve
from sklearn import metrics
from sklearn.utils import resample



In [3]:
no_nontoxic_nans_x_test = pd.read_csv('no_nontoxic_nans_x_test.csv')
no_nontoxic_nans_x_train = pd.read_csv('no_nontoxic_nans_x_train.csv')
no_nontoxic_nans_x_val = pd.read_csv('no_nontoxic_nans_x_val.csv')

no_nontoxic_nans_y_test = pd.read_csv('no_nontoxic_nans_y_test.csv')
no_nontoxic_nans_y_train = pd.read_csv('no_nontoxic_nans_y_train.csv')
no_nontoxic_nans_y_val = pd.read_csv('no_nontoxic_nans_y_val.csv')


,Unnamed: 0,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,3647,-1.477805,-1.477805,0.435245,0.850161,-1.226381,-0.911378,-1.223842,-1.212396,-1.226039,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,-0.124154,0.0,-0.102171,-0.259939,-0.163546
1,1817,0.119324,0.119324,-0.335013,0.212740,-0.020010,-0.315390,-0.970381,-0.992255,-0.969442,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,-0.124154,0.0,-0.102171,-0.259939,5.397008
2,4769,0.035269,0.035269,0.769528,0.044427,-0.595434,-0.590461,-0.735068,-0.675509,-0.739978,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,-0.124154,0.0,-0.102171,-0.259939,-0.163546
3,2070,0.038593,0.038593,3.984476,-2.441196,-1.366188,-0.525220,-0.459681,-0.370904,-0.458151,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,-0.124154,0.0,-0.102171,-0.259939,-0.163546
4,2424,0.406011,0.406011,-0.477332,0.582857,-0.269464,1.296834,-0.393832,-0.451158,-0.392890,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,-0.124154,0.0,-0.102171,-0.259939,-0.163546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,862,-0.242254,-0.242254,-0.608971,0.247389,0.263874,-0.045608,-0.496436,-0.559574,-0.495371,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,-0.124154,0.0,-0.102171,0.262915,-0.163546
1051,318,0.745946,0.745946,-0.459092,0.323580,-1.199935,-0.501636,1.154984,1.171752,1.156016,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,8.054502,0.0,-0.102171,-0.259939,-0.163546
1052,4118,0.143482,0.143482,-0.078210,-2.466516,0.181000,-0.390270,-0.253641,-0.221301,-0.252851,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,-0.124154,0.0,-0.102171,-0.259939,-0.163546
1053,108,0.569107,0.569107,-0.403218,0.387937,2.122048,-0.572123,0.145988,0.166908,0.144251,...,-0.167906,-0.165724,-0.069039,-0.09787,-0.043602,-0.124154,0.0,-0.102171,-0.259939,5.397008


In [ ]:
XGBR = XGBRegressor(eta=0.3, gamma=0, max_depth=6, subsample=1, min_child_weight=1)

XGBR_model = XGBR.fit(no_nontoxic_nans_x_train, no_nontoxic_nans_y_train)

XGBR_X_train = XGBR.predict(no_nontoxic_nans_x_train)
XGBR_X_val = XGBR.predict(no_nontoxic_nans_x_val)

XGBR_r2_train = metrics.r2_score(no_nontoxic_nans_y_train, XGBR_X_train)
XGBR_mae_train = metrics.mean_absolute_error(no_nontoxic_nans_y_train, XGBR_X_train)
XGBR_r2_val = metrics.r2_score(no_nontoxic_nans_y_val, XGBR_X_val)
XGBR_mae_val = metrics.mean_absolute_error(no_nontoxic_nans_y_val, XGBR_X_val)

print('XGBR')
print('R2 train:',XGBR_r2_train)
print('R2 val:',XGBR_r2_val)
print()
print('MAE train:',XGBR_mae_train)
print('MAE val:',XGBR_mae_val)

XGBR
R2 train: 0.9830904006958008
R2 val: 0.6341766119003296

MAE train: 0.9185823202133179
MAE val: 4.747739315032959


In [5]:
from xgboost import XGBClassifier
from sklearn import metrics

# 1. Define the classifier
XGBC = XGBClassifier(
    eta=0.3,
    gamma=0,
    max_depth=6,
    subsample=1,
    min_child_weight=1,
    objective='binary:logistic',   # binary classification
    eval_metric='logloss'          # or 'auc'
)

# 2. Fit model
XGBC_model = XGBC.fit(no_nontoxic_nans_x_train, no_nontoxic_nans_y_train['toxic'])

# 3. Predictions (class labels)
XGBC_y_train_pred = XGBC.predict(no_nontoxic_nans_x_train)
XGBC_y_val_pred   = XGBC.predict(no_nontoxic_nans_x_val)

# (Optional) predicted probabilities for ROC-AUC, PR curves, etc.
XGBC_y_train_proba = XGBC.predict_proba(no_nontoxic_nans_x_train)[:, 1]
XGBC_y_val_proba   = XGBC.predict_proba(no_nontoxic_nans_x_val)[:, 1]

# 4. Metrics (classification instead of regression)
XGBC_acc_train = metrics.accuracy_score(no_nontoxic_nans_y_train['toxic'], XGBC_y_train_pred)
XGBC_acc_val   = metrics.accuracy_score(no_nontoxic_nans_y_val['toxic'],   XGBC_y_val_pred)

XGBC_f1_train  = metrics.f1_score(no_nontoxic_nans_y_train['toxic'], XGBC_y_train_pred)
XGBC_f1_val    = metrics.f1_score(no_nontoxic_nans_y_val['toxic'],   XGBC_y_val_pred)

# If you want AUC (recommended for toxicity):
XGBC_auc_train = metrics.roc_auc_score(no_nontoxic_nans_y_train['toxic'], XGBC_y_train_proba)
XGBC_auc_val   = metrics.roc_auc_score(no_nontoxic_nans_y_val['toxic'],   XGBC_y_val_proba)

print('XGBC (classifier)')
print('Accuracy train:', XGBC_acc_train)
print('Accuracy val:  ', XGBC_acc_val)
print()
print('F1 train:', XGBC_f1_train)
print('F1 val:  ', XGBC_f1_val)
print()
print('ROC-AUC train:', XGBC_auc_train)
print('ROC-AUC val:  ', XGBC_auc_val)



XGBC (classifier)
Accuracy train: 0.9990521327014218
Accuracy val:   0.7630331753554502

F1 train: 0.9991199765327076
F1 val:   0.7771836007130125

ROC-AUC train: 0.999998593982244
ROC-AUC val:   0.842589852237373


,Unnamed: 0,toxic
0,1902,0
1,4184,0
2,569,0
3,3319,0
4,4665,0
...,...,...
3160,4269,0
3161,3281,1
3162,152,1
3163,4390,1


In [4]:
from sklearn.svm import SVC
from sklearn import metrics

# 1. Define the classifier
SVC_model = SVC(
    kernel='rbf',          # common, non-linear kernel
    C=1.0,                 # regularization strength
    class_weight='balanced',  # good if classes are imbalanced
    probability=True,      # enables predict_proba (needed for AUC)
    random_state=0
)

# 2. Fit model
SVC_model.fit(no_nontoxic_nans_x_train, y_train_bin)

# 3. Predictions (labels)
svc_y_train_pred = SVC_model.predict(no_nontoxic_nans_x_train)
svc_y_val_pred   = SVC_model.predict(no_nontoxic_nans_x_val)

# 4. Prediction probabilities (for ROC-AUC etc.)
svc_y_train_proba = SVC_model.predict_proba(no_nontoxic_nans_x_train)[:, 1]
svc_y_val_proba   = SVC_model.predict_proba(no_nontoxic_nans_x_val)[:, 1]

# 5. Metrics
svc_acc_train = metrics.accuracy_score(y_train_bin, svc_y_train_pred)
svc_acc_val   = metrics.accuracy_score(y_val_bin,   svc_y_val_pred)

svc_f1_train  = metrics.f1_score(y_train_bin, svc_y_train_pred)
svc_f1_val    = metrics.f1_score(y_val_bin,   svc_y_val_pred)

svc_auc_train = metrics.roc_auc_score(y_train_bin, svc_y_train_proba)
svc_auc_val   = metrics.roc_auc_score(y_val_bin,   svc_y_val_proba)

print("SVC (classifier)")
print("Accuracy train:", svc_acc_train)
print("Accuracy val:  ", svc_acc_val)
print()
print("F1 train:", svc_f1_train)
print("F1 val:  ", svc_f1_val)
print()
print("ROC-AUC train:", svc_auc_train)
print("ROC-AUC val:  ", svc_auc_val)

NameError: name 'y_train_bin' is not defined